In [16]:
import pandas as pd


data = pd.read_pickle('./reviews_processed_fixed.pkl')


In [17]:
dataTransposed = data.transpose()

dataTransposed.head()

,count,name,price,pricePer,rating,reviews,url,processedReviews
B079YF1K1B,120,"PhosphatidylSerine 300mg Per Serving, Made in ...",19.95,0.17,4.5,"[{'user': 'Beth and Dan Richard', 'rating': 1....",/PhosphatidylSerine-Phosphatidyl-Double-Wood-S...,"[[work, food, bought, three, different, bottle..."
B08XYDDXDC,120,"Nutricost Phosphatidylserine 400mg, 120 Capsul...",17.95,0.15,4.6,"[{'user': 'Barbara Greenan', 'rating': 1.0, 't...",/Nutricost-Phosphatidylserine-400mg-120-Capsul...,"[[improve, memory], [good, product]]"
B08LF5G6J1,90,Ultra High Purity Phosphatidylserine from Ital...,19.97,0.22,4.6,"[{'user': 'Michael K', 'rating': 5.0, 'text': ...",/Ultra-High-Purity-Phosphatidylserine-Italy-St...,"[[partner, second, bottle, impressed, change, ..."
B0013OXFTU,120,"NOW Supplemnets, Phosphatidyl Serine 100 mg wi...",31.95,0.27,4.6,"[{'user': 'dSavannah', 'rating': 5.0, 'text': ...",/NOW-Phosphatidyl-Serine-100mg-Capsules/dp/B00...,"[[take, tried, variety, supplement, medication..."
B0013OXA90,60,Jarrow Formulas PS 100-60 Softgels - 100 mg Ph...,21.57,0.36,4.6,"[{'user': 'kswen', 'rating': 5.0, 'text': 'I'm...",/Jarrow-Formulas-Supports-Cognitive-Function/d...,"[[impressed, supplement, changed, quality, sle..."


In [18]:
processedReviewsDF = dataTransposed['processedReviews']

arr = dataTransposed['processedReviews'].tolist()

arr_flat = []
for e1 in arr:
    if e1:
        for e2 in e1:
            if e2:
                arr_flat.append(e2)


In [19]:
import gensim


dictionary = gensim.corpora.Dictionary(arr_flat)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


0 amazon
1 blocked
2 bottle
3 bought
4 brand
5 corrected
6 dan
7 different
8 directly
9 eurofins
10 food


In [20]:
# Filter Tokens

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

In [21]:
# Create bag of words

bow_corpus = [dictionary.doc2bow(doc) for doc in arr_flat]

# Check to see preview of Bag of Words

bow_doc_20 = bow_corpus[20]

for i in range(len(bow_doc_20)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_20[i][0], 
                                               dictionary[bow_doc_20[i][0]], 
bow_doc_20[i][1]))

Word 7 ("lot") appears 1 time.
Word 18 ("three") appears 1 time.
Word 19 ("two") appears 1 time.
Word 22 ("also") appears 1 time.
Word 29 ("day") appears 1 time.
Word 42 ("take") appears 1 time.
Word 63 ("supplement") appears 1 time.
Word 86 ("good") appears 1 time.
Word 95 ("noticed") appears 1 time.
Word 100 ("taking") appears 2 time.
Word 126 ("went") appears 1 time.
Word 130 ("big") appears 1 time.
Word 141 ("medication") appears 1 time.
Word 151 ("thing") appears 1 time.
Word 207 ("started") appears 1 time.
Word 292 ("think") appears 1 time.
Word 318 ("difference") appears 1 time.
Word 321 ("child") appears 1 time.
Word 322 ("clearly") appears 1 time.
Word 323 ("dream") appears 1 time.
Word 324 ("kind") appears 1 time.
Word 325 ("plan") appears 1 time.
Word 326 ("remember") appears 1 time.
Word 327 ("start") appears 1 time.


In [22]:
#Create tf-idf model on bow_corpus

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [24]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"day" + 0.024*"product" + 0.015*"work" + 0.015*"take" + 0.015*"better" + 0.014*"help" + 0.013*"focus" + 0.013*"brain" + 0.012*"year" + 0.011*"supplement"
Topic: 1 
Words: 0.036*"fish" + 0.033*"omega" + 0.029*"oil" + 0.028*"product" + 0.027*"good" + 0.023*"brand" + 0.022*"quality" + 0.019*"fishy" + 0.019*"great" + 0.018*"bottle"
Topic: 2 
Words: 0.084*"product" + 0.074*"great" + 0.026*"memory" + 0.021*"work" + 0.017*"focus" + 0.017*"taking" + 0.015*"help" + 0.015*"feel" + 0.014*"using" + 0.013*"supplement"
Topic: 3 
Words: 0.051*"sleep" + 0.033*"night" + 0.025*"eye" + 0.021*"easy" + 0.021*"taste" + 0.018*"swallow" + 0.018*"taking" + 0.017*"better" + 0.016*"lemon" + 0.016*"help"
Topic: 4 
Words: 0.035*"fish" + 0.031*"product" + 0.029*"help" + 0.025*"oil" + 0.024*"nordic" + 0.018*"burp" + 0.015*"recommend" + 0.013*"taste" + 0.012*"tried" + 0.012*"day"
Topic: 5 
Words: 0.038*"product" + 0.025*"good" + 0.024*"taking" + 0.018*"cortisol" + 0.018*"feel" + 0.016*"level" +

In [25]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/CSUser/Documents/NLP/Project/Project/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein packa

Topic: 0 Word: 0.075*"good" + 0.023*"product" + 0.014*"stuff" + 0.014*"best" + 0.013*"far" + 0.011*"eye" + 0.011*"great" + 0.010*"price" + 0.009*"expected" + 0.009*"like"
Topic: 1 Word: 0.080*"great" + 0.044*"product" + 0.035*"burp" + 0.027*"excellent" + 0.022*"work" + 0.018*"fish" + 0.011*"taste" + 0.010*"joint" + 0.010*"love" + 0.010*"buy"
Topic: 2 Word: 0.025*"love" + 0.017*"help" + 0.015*"product" + 0.014*"using" + 0.013*"blood" + 0.011*"recommended" + 0.011*"memory" + 0.010*"year" + 0.010*"doctor" + 0.010*"sleep"
Topic: 3 Word: 0.031*"easy" + 0.030*"swallow" + 0.018*"eye" + 0.014*"taste" + 0.012*"health" + 0.011*"take" + 0.011*"capsule" + 0.010*"recommended" + 0.010*"product" + 0.010*"dry"
Topic: 4 Word: 0.035*"fish" + 0.028*"oil" + 0.011*"happy" + 0.011*"product" + 0.010*"lemon" + 0.010*"great" + 0.010*"purchase" + 0.009*"ok" + 0.009*"taste" + 0.009*"best"
Topic: 5 Word: 0.016*"feel" + 0.013*"better" + 0.012*"focus" + 0.012*"product" + 0.012*"day" + 0.011*"difference" + 0.011*"he

In [42]:
for index, score in sorted(lda_model_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))



Score: 0.9709568619728088	 
Topic: 0.049*"quality" + 0.016*"lemon" + 0.016*"product" + 0.015*"good" + 0.014*"high" + 0.012*"great" + 0.012*"taste" + 0.012*"advertised" + 0.010*"oil" + 0.009*"fish"


In [43]:
for index, score in sorted(lda_model_tfidf[bow_corpus[19610]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))



Score: 0.8874669075012207	 
Topic: 0.080*"great" + 0.044*"product" + 0.035*"burp" + 0.027*"excellent" + 0.022*"work" + 0.018*"fish" + 0.011*"taste" + 0.010*"joint" + 0.010*"love" + 0.010*"buy"

Score: 0.012504358775913715	 
Topic: 0.049*"quality" + 0.016*"lemon" + 0.016*"product" + 0.015*"good" + 0.014*"high" + 0.012*"great" + 0.012*"taste" + 0.012*"advertised" + 0.010*"oil" + 0.009*"fish"

Score: 0.012504163198173046	 
Topic: 0.020*"nordic" + 0.019*"aftertaste" + 0.019*"like" + 0.018*"taste" + 0.017*"fishy" + 0.014*"cholesterol" + 0.013*"natural" + 0.012*"product" + 0.008*"year" + 0.007*"take"

Score: 0.01250413991510868	 
Topic: 0.025*"love" + 0.017*"help" + 0.015*"product" + 0.014*"using" + 0.013*"blood" + 0.011*"recommended" + 0.011*"memory" + 0.010*"year" + 0.010*"doctor" + 0.010*"sleep"

Score: 0.012504053302109241	 
Topic: 0.075*"good" + 0.023*"product" + 0.014*"stuff" + 0.014*"best" + 0.013*"far" + 0.011*"eye" + 0.011*"great" + 0.010*"price" + 0.009*"expected" + 0.009*"like"

